<a href="https://colab.research.google.com/github/LaraSaads/Projects/blob/main/loan_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn. model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
df=pd.read_csv('/content/train Loan prediction.csv')
df = df.drop(['Loan_ID'], axis=1)
for gender in df['Gender']:
  if gender=='Male':
    df['Gender']=0
  else:
    df['Gender']=1
encoder=LabelEncoder()
columns_to_encode = ['Married', 'Dependents', 'Education', 'Self_Employed', 'Loan_Status']

for col in columns_to_encode:
  df[col] = encoder.fit_transform(df[col])
encoder = LabelEncoder()

df['Property_Area'] = encoder.fit_transform(df['Property_Area'])
df.fillna(0, inplace=True)
cols=['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']
scaler=StandardScaler()
df[cols]=scaler.fit_transform(df[cols])
x=df.drop(columns=['Loan_Status'])
y=df['Loan_Status']

print("Top correlations with Loan Status:")
print(df.corr()["Loan_Status"].abs().sort_values(ascending=False).head(11))

selector = SelectKBest(f_classif, k=min(11, x.shape[1]))
selector.fit(x, y)
selected_features = x.columns[selector.get_support()].tolist()
print("Selected features:", selected_features)

x_selected = selector.transform(x)

Top correlations with Loan Status:
Loan_Status          1.000000
Credit_History       0.432616
Married              0.096657
Education            0.085884
CoapplicantIncome    0.059187
Property_Area        0.032112
Self_Employed        0.010880
LoanAmount           0.010631
Dependents           0.007318
Loan_Amount_Term     0.007306
ApplicantIncome      0.004710
Name: Loan_Status, dtype: float64
Selected features: ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x_selected, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
model = Sequential([
    Dense(64, input_dim=x_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(x_train, y_train,
                    validation_split=0.2,
                    epochs=50,
                    batch_size=32,
                    verbose=1)


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.5846 - loss: 0.6803 - val_accuracy: 0.6970 - val_loss: 0.6229
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6795 - loss: 0.6601 - val_accuracy: 0.6970 - val_loss: 0.6121
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7137 - loss: 0.6108 - val_accuracy: 0.6970 - val_loss: 0.6066
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6933 - loss: 0.6235 - val_accuracy: 0.6970 - val_loss: 0.6062
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6562 - loss: 0.6427 - val_accuracy: 0.6970 - val_loss: 0.6010
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6670 - loss: 0.6453 - val_accuracy: 0.6970 - val_loss: 0.5973
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7102 - loss: 0.5816 - val_accuracy: 0.6869 - val_loss: 0.5941
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7059 - loss: 0.5868 - val_accuracy: 0.6869 - val_loss

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7921 - loss: 0.4902 
Test Accuracy: 0.7642


In [ ]:
pred=model.predict(x_test)
Predicition=(pred>=0.5).astype(int)
comp=pd.DataFrame({'Actual':y_test,'Predicted':Predicition.flatten()})
print(comp)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
     Actual  Predicted
150       0          0
559       1          1
598       1          1
235       1          1
145       1          1
..      ...        ...
363       1          0
280       0          0
7         0          0
289       1          1
373       0          0

[123 rows x 2 columns]
